In [1]:
# read an xlx file in pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read Author info, which contains all the pairs
paper_auth_pairs = pd.read_excel('input_data/2025-02-14_last_xlsx/1_Triage_Last author.xlsx', sheet_name='Tri sans les doublons')
# Drop all columns with 'Unnamed' in the name
paper_auth_pairs = paper_auth_pairs.drop(columns=[col for col in paper_auth_pairs.columns if 'Unnamed' in col]).drop(columns=['Source'])
paper_auth_pairs.to_csv('input_data/2025-02-14_last_xlsx/1_Triage_Last author.csv', index=False)

first_authors_claims = pd.read_excel('input_data/2025-02-14_last_xlsx/stats_author.xlsx', sheet_name='First')
leading_authors_claims = pd.read_excel('input_data/2025-02-14_last_xlsx/stats_author.xlsx', sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"


authors_claims = pd.concat([leading_authors_claims, first_authors_claims])
authors_claims['Sex'] = authors_claims['Sex'].map({1: 'Male', 0: 'Female'})
authors_claims = authors_claims.drop(columns=[col for col in authors_claims.columns if '%' in col])
authors_claims.rename(columns={'Conituinity': 'Continuity'}, inplace=True)
authors_claims['Historical lab'] = authors_claims['Historical lab'].astype('boolean')
authors_claims['Continuity'] = authors_claims['Continuity'].astype('boolean')
authors_claims["Partially Verified"] = authors_claims["Partially verified"]
authors_claims = authors_claims.drop(columns=["Partially verified"])
authors_claims.to_csv('input_data/2025-02-14_last_xlsx/stats_author.csv', index=False)


In [2]:
paper_auth_pairs



,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league
0,Agaisse H,Derré,0,Post-doc,1,PI,0,Yale University,USA,1
1,Aguilera RJ,Seong CS,1,PhD,0,Academia,0,The University of Texas,USA,0
2,Aigaki T,Tsuda M,1,PhD,1,Admin,0,Tokyo Metropolitan University,Japan,0
3,Ando,Markus,1,PhD,0,Facility leader,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
4,Ando,Rus F,0,PhD,0,??,0,"Hungarian Academy of Sciences, Szeged",Hungary,0
...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,??,PhD,??,Senior Staff,0,niversity of Missouri-Kansas City,USA,0
316,Zhu S,Yuan Y,1,PhD,0,Senior Staff,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
317,Zhu S,Tian C,0,PhD,1,PI,0,"Institute of Zoology, Chinese Academy of Sciences",China,0
318,Zhu S,Zhang Z,0,PhD,0,Academia,0,"Institute of Zoology, Chinese Academy of Sciences",China,0


 It seems that

 - sex -> FH

 - PhD Post-doc -> FH

 - Become a Pi -> FH

 - current job -> FH

 - MD -> **???**

 - Affiliation -> Both

 - Country -> Both

 - Ivy League -> Both

In [3]:
def deduplicate_by(df, col_name):
    """
    Deduplicate a dataframe based on a specific column, keeping the most common values 
    for other columns when duplicates exist.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe to deduplicate
    col_name : str
        The column name to deduplicate by
        
    Returns:
    --------
    pandas.DataFrame
        Deduplicated dataframe with one row per unique value in col_name
    """
    from collections import Counter
    import pandas as pd
    import numpy as np
    
    # Create a list to store unique values and their most common attribute values
    unique_rows = []
    
    # Get unique values in the specified column
    unique_values = df[col_name].unique()
    
    # For each unique value
    for value in unique_values:
        # Get all rows with this value
        value_rows = df[df[col_name] == value]
        
        # Initialize a row for this unique value
        unique_row = {col_name: value}
        
        # For each column except the one we're deduplicating by
        for col in df.columns:
            if col == col_name:
                continue
                
            # Get the most common value
            values = value_rows[col].dropna().tolist()
            if len(values) == 0:
                unique_row[col] = np.nan
                continue
                
            # Use Counter to find the most common value
            value_counts = Counter(values)
            most_common_value, count = value_counts.most_common(1)[0]
            
            # Check if there are ties for most common value
            if sum(1 for v, c in value_counts.items() if c == count) > 1:
                print(f"Warning: Multiple most common values for {value} in column {col}. Choosing {most_common_value}")
            
            unique_row[col] = most_common_value
        
        unique_rows.append(unique_row)
    
    # Create a new DataFrame from the unique values
    result_df = pd.DataFrame(unique_rows)
    
    # Reorder columns to match original DataFrame
    result_df = result_df[df.columns]
    
    return result_df



In [4]:
paper_auth_pairs_LH = paper_auth_pairs[["last author", "Affiliation", "Country", "Ivy league"]]
paper_auth_pairs_LH = deduplicate_by(paper_auth_pairs_LH, "last author")
claims_LH = authors_claims[authors_claims['Authorship'] == 'Leading']


In [5]:
paper_auth_pairs_FH = paper_auth_pairs[["first author", "Affiliation", "Country", "Ivy league"]] # TODO
paper_auth_pairs_FH = deduplicate_by(paper_auth_pairs_FH, "first author")
claims_FH = authors_claims[authors_claims['Authorship'] == 'First']

# create merge columns: lowercased and stripped of accents
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['first author'].str.lower()
paper_auth_pairs_FH['fh_proc'] = paper_auth_pairs_FH['fh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
claims_FH['fh_proc'] = claims_FH['Name'].str.lower()
claims_FH['fh_proc'] = claims_FH['fh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

all_FH = pd.merge(claims_FH, paper_auth_pairs_FH, on='fh_proc', how='outer')
print(len(claims_FH), len(paper_auth_pairs_FH), len(all_FH))

unique_pairs = all_FH[["Name", "first author", "fh_proc"]].drop_duplicates().sort_values("first author", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['first author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['fh_proc']:<20} {unique_pairs.iloc[i]['first author']:<20}  {unique_pairs.iloc[i]['Name']}")


293 291 297
💥 derre                Derré                 nan
💥 markus               Markus                nan
💥 rizki mt             RIZKI MT              nan
💥 schneider d          Schneider D           nan
💥 markus r             nan                   Márkus R
💥 hedengren-olcott m   nan                   Hedengren-Olcott M
💥 derre i              nan                   Derré I
💥 bellotti ra          nan                   Bellotti RA


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_32517/3122261078.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_FH['fh_proc'] = claims_FH['Name'].str.lower()
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_32517/3122261078.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_FH['fh_proc'] = claims_FH['fh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [6]:
# create merge columns: lowercased and stripped of accents
paper_auth_pairs_LH['lh_proc'] = paper_auth_pairs_LH['last author'].str.lower()
paper_auth_pairs_LH['lh_proc'] = paper_auth_pairs_LH['lh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
claims_LH['lh_proc'] = claims_LH['Name'].str.lower()
claims_LH['lh_proc'] = claims_LH['lh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# replace ando i by ando
claims_LH['lh_proc'] = claims_LH['lh_proc'].str.replace('ando i', 'ando')

all_LH = pd.merge(claims_LH, paper_auth_pairs_LH, on='lh_proc', how='outer')
print(len(claims_LH), len(paper_auth_pairs_LH), len(all_LH))


152 161 163


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_32517/2810914933.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_LH['lh_proc'] = claims_LH['Name'].str.lower()
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_32517/2810914933.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_LH['lh_proc'] = claims_LH['lh_proc'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_32517/2810914933.py:7

In [7]:
unique_pairs = all_LH[["Name", "last author", "lh_proc"]].drop_duplicates().sort_values("last author", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['last author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['lh_proc']:<20} {unique_pairs.iloc[i]['last author']:<20}  {unique_pairs.iloc[i]['Name']}")


💥 bellotti ra          Bellotti RA           nan
💥 boccard f            Boccard F             nan
💥 brey pt              Brey PT               nan
💥 cerenius l           Cerenius L            nan
💥 higgins de           Higgins DE            nan
💥 mengin-lecreulx d    Mengin-Lecreulx D     nan
💥 moore kj             Moore KJ              nan
💥 rizki mt             RIZKI MT              nan
💥 shahabuddin m        Shahabuddin M         nan
💥 shirasu-hiza mm      Shirasu-Hiza MM       nan
💥 silvers mj           Silvers MJ            nan
💥 matova n             nan                   Matova N
💥 nappi aj             nan                   Nappi AJ


In [8]:
all_LH_inner = pd.merge(claims_LH, paper_auth_pairs_LH, on='lh_proc', how='inner')
print(len(all_LH_inner))


150


 ## Let's go for last authors first

 There are two ways to do that. By paper or by author. Perhaps by author ?

In [9]:
print(all_LH_inner.head())
df = all_LH_inner


           Name   Sex  Historical lab  Continuity  Articles  Major claims  \
0    Lemaitre B  Male            True        True        24            69   
1    Hultmark D  Male            True        True        28            67   
2   Hoffmann JA  Male            True        True        16            45   
3  Schneider DS  Male           False       False        12            31   
4        Lee WJ  Male           False        True        12            31   

   Unchallenged  Verified  Mixed  Challenged Authorship  Partially Verified  \
0             8        55      0           1    Leading                   5   
1            12        47      0           2    Leading                   6   
2             2        39      0           2    Leading                   2   
3            17         6      0           7    Leading                   1   
4             8        14      2           4    Leading                   3   

        lh_proc   last author                                 

In [10]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.patches as mpatches

from analysis_claims import ASSESSMENT_COLORS


# Set global plotting parameters
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


ModuleNotFoundError: No module named 'analysis_claims'

In [ ]:
# Let's assume df is already loaded
# First, let's create proportions for each assessment type for each PI
assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']

for col in assessment_columns:
    df[f'{col}_prop'] = df[col] / df['Major claims']

# Let's also create a "reproducibility score" - higher means more verified claims
df['reproducibility_score'] = (df['Verified_prop'] * 2 + df['Partially Verified_prop'] - 
                              df['Challenged_prop'] * 2 - df['Mixed_prop'] * 0.5)

# Let's examine the distribution of this score
plt.figure(figsize=(10, 6))
sns.histplot(df['reproducibility_score'], kde=True)
plt.title('Distribution of Reproducibility Scores')
plt.xlabel('Reproducibility Score')
plt.axvline(df['reproducibility_score'].mean(), color='red', linestyle='--', 
            label=f'Mean = {df["reproducibility_score"].mean():.2f}')
plt.legend()
plt.show()

# Basic descriptive statistics
descriptive_stats = df[assessment_columns + [col + '_prop' for col in assessment_columns] + 
                     ['reproducibility_score', 'Articles', 'Major claims']].describe()
print("Descriptive Statistics:")
descriptive_stats


In [ ]:
ASSESSMENT_COLORS


In [ ]:
ASSESSMENT_COLORS[category]


In [ ]:
# Group by historical lab status
historical_grouped = df.groupby('Historical lab').agg({
    **{col: 'sum' for col in assessment_columns},
    'Major claims': 'sum',
    'Articles': 'sum',
    'reproducibility_score': 'mean'
})

# Calculate proportions
for col in assessment_columns:
    historical_grouped[f'{col}_prop'] = historical_grouped[col] / historical_grouped['Major claims']

# Create stacked bar chart
fig, ax = plt.subplots(figsize=(12, 8))

# Prepare data for stacking
data = []
categories = []
for col in ['Verified_prop', 'Partially Verified_prop', 'Unchallenged_prop', 'Mixed_prop', 'Challenged_prop']:
    data.append(historical_grouped[col].values)
    categories.append(col.replace('_prop', ''))

# Create stacked bars
x = np.arange(len(historical_grouped.index))
bottom = np.zeros(len(historical_grouped.index))
bars = []

for i, category in enumerate(categories):
    bar = ax.bar(x, data[i], bottom=bottom, label=category, 
                color=ASSESSMENT_COLORS[category])
    bottom += data[i]
    bars.append(bar)

# Add data labels to each segment
for bar in bars:
    for rect in bar:
        height = rect.get_height()
        if height > 0.05:  # Only add label if segment is large enough
            ax.text(rect.get_x() + rect.get_width()/2., rect.get_y() + height/2.,
                  f'{height:.2f}', ha='center', va='center', color='white', fontweight='bold')

# Add number of papers as text on bars
for i, lab in enumerate(historical_grouped.index):
    ax.text(i, 1.02, f"n={historical_grouped.loc[lab, 'Articles']}", ha='center')

# Customize plot
ax.set_xticks(x)
ax.set_xticklabels(['Non-traditional Labs', 'Traditional-Historical Labs'])
ax.set_ylabel('Proportion of Claims')
ax.set_title('Assessment of Scientific Claims by Lab Tradition')
ax.legend(loc='upper right')

# Add statistical annotation
t_stat, p_val = stats.ttest_ind(
    df[df['Historical lab'] == True]['reproducibility_score'].dropna(),
    df[df['Historical lab'] == False]['reproducibility_score'].dropna(),
    equal_var=False
)

sign = "*" if p_val < 0.05 else "ns"
ax.text(0.5, 1.08, f"Reproducibility score difference: t={t_stat:.2f}, p={p_val:.3f} {sign}", 
      ha='center', transform=ax.transAxes, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary
print("Summary of Traditional vs. Non-traditional Labs:")
print(historical_grouped[['Major claims', 'Articles', 'reproducibility_score', 
                        'Verified_prop', 'Challenged_prop', 'Unchallenged_prop']])


In [ ]:
# Group by country and calculate proportions
country_grouped = df.groupby('Country').agg({
    **{col: 'sum' for col in assessment_columns},
    'Major claims': 'sum',
    'Articles': 'count',
    'reproducibility_score': 'mean'
}).reset_index()

# Calculate proportions
for col in assessment_columns:
    country_grouped[f'{col}_prop'] = country_grouped[col] / country_grouped['Major claims']

# Filter to include only countries with sufficient data (at least 5 PIs)
country_filtered = country_grouped[country_grouped['Articles'] >= 5]

# Sort by reproducibility score
country_filtered = country_filtered.sort_values('reproducibility_score', ascending=False)

# Create a horizontal grouped bar chart
fig, ax = plt.subplots(figsize=(14, 10))

# Set width and positions
width = 0.15
x = np.arange(len(country_filtered))

# Plot each assessment type
for i, category in enumerate(['Verified_prop', 'Partially Verified_prop', 'Unchallenged_prop', 
                             'Mixed_prop', 'Challenged_prop']):
    ax.barh(x + i*width - 0.3, country_filtered[category], width, 
           label=category.replace('_prop', ''),
           color=ASSESSMENT_COLORS[category.replace('_prop', '')])

# Customize plot
ax.set_yticks(x)
ax.set_yticklabels(country_filtered['Country'])
ax.set_xlabel('Proportion of Claims')
ax.set_title('Scientific Claim Assessment by Country')
ax.legend(loc='upper right')

# Add sample size annotation
for i, country in enumerate(country_filtered['Country']):
    ax.text(0, i - 0.4, f"n={country_filtered.iloc[i]['Articles']} PIs, {country_filtered.iloc[i]['Major claims']} claims", 
           fontsize=10)

# Add ANOVA results for reproducibility score differences by country
f_stat, p_val = stats.f_oneway(
    *[df[df['Country'] == country]['reproducibility_score'].dropna() 
     for country in country_filtered['Country']]
)

ax.text(0.5, -0.08, f"ANOVA for reproducibility score by country: F={f_stat:.2f}, p={p_val:.3f}", 
      ha='center', transform=ax.transAxes, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Print numerical summary
print("Country Rankings by Reproducibility Score:")
print(country_filtered[['Country', 'Articles', 'Major claims', 'reproducibility_score',
                       'Verified_prop', 'Challenged_prop', 'Unchallenged_prop']])


In [ ]:
# Group by continuity in the field
continuity_grouped = df.groupby('Continuity').agg({
    **{col: 'sum' for col in assessment_columns},
    'Major claims': 'sum',
    'Articles': 'sum',
    'reproducibility_score': 'mean'
})

# Calculate proportions
for col in assessment_columns:
    continuity_grouped[f'{col}_prop'] = continuity_grouped[col] / continuity_grouped['Major claims']

# Create a radar chart to compare the profiles
categories = ['Verified_prop', 'Partially Verified_prop', 'Unchallenged_prop', 
             'Mixed_prop', 'Challenged_prop']
categories_clean = [cat.replace('_prop', '') for cat in categories]

# Number of variables
N = len(categories)

# What will be the angle of each axis
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Close the loop

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

# Draw one axis per variable + add labels
plt.xticks(angles[:-1], categories_clean, size=14)

# Draw the data for established researchers
values_true = continuity_grouped.loc[True, categories].values.flatten().tolist()
values_true += values_true[:1]  # Close the loop
ax.plot(angles, values_true, linewidth=3, linestyle='solid', 
       label='Established Researchers', color='#2ecc71')
ax.fill(angles, values_true, alpha=0.25, color='#2ecc71')

# Draw the data for newcomers
values_false = continuity_grouped.loc[False, categories].values.flatten().tolist()
values_false += values_false[:1]  # Close the loop
ax.plot(angles, values_false, linewidth=3, linestyle='dashed', 
       label='Newcomers', color='#e74c3c')
ax.fill(angles, values_false, alpha=0.25, color='#e74c3c')

# Add legend and title
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.title('Claim Assessment Profile: Newcomers vs. Established Researchers', size=18, y=1.1)

# Customize radar chart
ax.set_ylim(0, max(max(values_true), max(values_false)) * 1.1)
ax.grid(True, alpha=0.3)

# Add sample size annotation
plt.annotate(f"Established: n={continuity_grouped.loc[True, 'Articles']} articles, {continuity_grouped.loc[True, 'Major claims']} claims", 
            xy=(0.5, -0.1), xycoords='axes fraction', ha='center')
plt.annotate(f"Newcomers: n={continuity_grouped.loc[False, 'Articles']} articles, {continuity_grouped.loc[False, 'Major claims']} claims", 
            xy=(0.5, -0.15), xycoords='axes fraction', ha='center')

# Add t-test results
t_stat, p_val = stats.ttest_ind(
    df[df['Continuity'] == True]['reproducibility_score'].dropna(),
    df[df['Continuity'] == False]['reproducibility_score'].dropna()
)
plt.annotate(f"Reproducibility score t-test: t={t_stat:.2f}, p={p_val:.3f}", 
            xy=(0.5, -0.2), xycoords='axes fraction', ha='center')

plt.tight_layout()
plt.show()

# Create a second visualization showing differences in unchallenged claims
fig, ax = plt.subplots(figsize=(10, 6))

# Bar chart for unchallenged claims
unchallenged_data = [continuity_grouped.loc[False, 'Unchallenged_prop'], 
                    continuity_grouped.loc[True, 'Unchallenged_prop']]
challenged_data = [continuity_grouped.loc[False, 'Challenged_prop'], 
                  continuity_grouped.loc[True, 'Challenged_prop']]

x = np.arange(2)
width = 0.35

unchallenged_bars = ax.bar(x - width/2, unchallenged_data, width, 
                         label='Unchallenged', color=ASSESSMENT_COLORS['Unchallenged'])
challenged_bars = ax.bar(x + width/2, challenged_data, width, 
                       label='Challenged', color=ASSESSMENT_COLORS['Challenged'])

# Add data labels
for bars in [unchallenged_bars, challenged_bars]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
               f'{height:.2f}', ha='center', va='bottom')

# Customize plot
ax.set_xticks(x)
ax.set_xticklabels(['Newcomers', 'Established Researchers'])
ax.set_ylabel('Proportion of Claims')
ax.set_title('Unchallenged vs. Challenged Claims by Field Experience')
ax.legend()

# Add statistical annotation
t_stat_unch, p_val_unch = stats.ttest_ind(
    df[df['Continuity'] == False]['Unchallenged_prop'].dropna(),
    df[df['Continuity'] == True]['Unchallenged_prop'].dropna()
)
ax.text(0.5, 0.95, f"Unchallenged proportion t-test: t={t_stat_unch:.2f}, p={p_val_unch:.3f}", 
       ha='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

# Print summary
print("Summary of Newcomers vs. Established Researchers:")
continuity_summary = continuity_grouped.copy()
continuity_summary.index = ['Newcomers', 'Established Researchers']
print(continuity_summary[['Major claims', 'Articles', 'reproducibility_score', 
                         'Verified_prop', 'Challenged_prop', 'Unchallenged_prop']])


In [ ]:
# Create a multivariable regression model to predict reproducibility
# First, ensure no object datatypes are in our dataset by examining the data
print("Data types before preprocessing:")
print(df[['Historical lab', 'Continuity', 'Ivy league', 'Articles', 
          'Sex', 'Unchallenged_prop', 'Challenged_prop', 'reproducibility_score']].dtypes)

# Convert any object columns to appropriate types
# Make sure all numeric columns are properly formatted
numeric_cols = ['Articles', 'Unchallenged_prop', 'Challenged_prop', 'reproducibility_score']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert boolean columns to integer
bool_cols = ['Historical lab', 'Continuity']
for col in bool_cols:
    df[col] = df[col].astype(int)
    
# Make sure categorical variables are properly typed
if 'Sex' in df.columns:
    df['Sex'] = df['Sex'].astype('category')
    
# Create dummy variables properly
df_model = pd.get_dummies(df[['Historical lab', 'Continuity', 'Ivy league', 
                             'Articles', 'Unchallenged_prop', 'Challenged_prop', 
                             'reproducibility_score']])

# Print data types after preprocessing
print("Data types after preprocessing:")
print(df_model.dtypes)

# Fit regression model for unchallenged proportion
# Select predictors (X) and target (y)
y = df_model['Unchallenged_prop']
X = df_model.drop(['Unchallenged_prop', 'Challenged_prop', 'reproducibility_score'], axis=1)

# Add constant
X = sm.add_constant(X)

# Verify no object datatypes remain
print("X dtypes:", X.dtypes.unique())
print("y dtype:", y.dtype)

# Fit model
try:
    model_unchallenged = sm.OLS(y, X).fit()
    print("Model successfully fit!")
    
    # Create another model for reproducibility score
    y2 = df_model['reproducibility_score']
    model_repro = sm.OLS(y2, X).fit()
    
    # Print summaries
    print("\nRegression Model for Unchallenged Proportion:")
    print(model_unchallenged.summary())
    
    print("\nRegression Model for Reproducibility Score:")
    print(model_repro.summary())
    
    # Create a visual summary of the regression results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Extract coefficients and confidence intervals for unchallenged model
    coefs1 = model_unchallenged.params[1:]
    conf_intervals1 = model_unchallenged.conf_int().iloc[1:]
    errors1 = (conf_intervals1[1] - conf_intervals1[0]) / 3.92  # 95% CI to standard error
    
    # Plot coefficients for unchallenged model
    ax1.errorbar(coefs1, range(len(coefs1)), xerr=errors1, fmt='o', capsize=5, color='#3498db')
    ax1.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    ax1.set_yticks(range(len(coefs1)))
    ax1.set_yticklabels(coefs1.index)
    ax1.set_xlabel('Coefficient Value')
    ax1.set_title('Predictors of Unchallenged Claims')
    
    # Extract coefficients and confidence intervals for reproducibility model
    coefs2 = model_repro.params[1:]
    conf_intervals2 = model_repro.conf_int().iloc[1:]
    errors2 = (conf_intervals2[1] - conf_intervals2[0]) / 3.92  # 95% CI to standard error
    
    # Plot coefficients for reproducibility model
    ax2.errorbar(coefs2, range(len(coefs2)), xerr=errors2, fmt='o', capsize=5, color='#2ecc71')
    ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    ax2.set_yticks(range(len(coefs2)))
    ax2.set_yticklabels(coefs2.index)
    ax2.set_xlabel('Coefficient Value')
    ax2.set_title('Predictors of Reproducibility Score')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error fitting model: {e}")
    print("Let's try an alternative approach using a simpler model")
    
    # Create a simpler model with fewer variables
    basic_vars = ['Articles', 'Historical lab', 'Continuity']
    df_simple = df[basic_vars + ['Unchallenged_prop']].copy()
    
    for col in basic_vars + ['Unchallenged_prop']:
        df_simple[col] = pd.to_numeric(df_simple[col], errors='coerce')
    
    # Drop missing values
    df_simple = df_simple.dropna()
    
    # Run simpler regression
    X_simple = sm.add_constant(df_simple[basic_vars])
    y_simple = df_simple['Unchallenged_prop']
    
    model_simple = sm.OLS(y_simple, X_simple).fit()
    print("\nSimplified Regression Model:")
    print(model_simple.summary())


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Read the data
df = pd.read_csv('test.csv')

# Calculate percentage of challenged claims for each author
df['challenged_rate'] = df['Challenged'] / df['Major claims'] * 100

# Function for statistical testing
def compare_groups(data, column, rate_column='challenged_rate'):
    groups = data[column].unique()
    if len(groups) == 2:  # For binary variables like Sex
        group1 = data[data[column] == groups[0]][rate_column]
        group2 = data[data[column] == groups[1]][rate_column]
        stat, pval = stats.mannwhitneyu(group1, group2)
        return {
            'groups': groups,
            'medians': [group1.median(), group2.median()],
            'p_value': pval
        }
    else:  # For variables with more than 2 categories
        stat, pval = stats.kruskal(*[group[rate_column].values 
                                    for name, group in data.groupby(column)])
        return {
            'groups': groups,
            'medians': [group[rate_column].median() 
                       for name, group in data.groupby(column)],
            'p_value': pval
        }

# Create subplots for our analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
plt.subplots_adjust(hspace=0.3)

# 1. Sex Analysis
sns.boxplot(data=df, x='Sex', y='challenged_rate', ax=axes[0,0])
sex_stats = compare_groups(df, 'Sex')
axes[0,0].set_title(f'Challenged Claims Rate by Sex\np={sex_stats["p_value"]:.3f}')

# 2. Historical Lab Analysis
sns.boxplot(data=df, x='Historical lab', y='challenged_rate', ax=axes[0,1])
lab_stats = compare_groups(df, 'Historical lab')
axes[0,1].set_title(f'Challenged Claims Rate by Historical Lab\np={lab_stats["p_value"]:.3f}')

# 3. Country Analysis
sns.boxplot(data=df, x='Country', y='challenged_rate', ax=axes[1,0])
country_stats = compare_groups(df, 'Country')
axes[1,0].set_title(f'Challenged Claims Rate by Country\np={country_stats["p_value"]:.3f}')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Ivy League vs Non-Ivy League
sns.boxplot(data=df, x='Ivy league', y='challenged_rate', ax=axes[1,1])
ivy_stats = compare_groups(df, 'Ivy league')
axes[1,1].set_title(f'Challenged Claims Rate by Ivy League Status\np={ivy_stats["p_value"]:.3f}')

# Add overall title
plt.suptitle('Factors Affecting Rate of Challenged Claims', fontsize=16, y=1.02)

# Print summary statistics
print("\nSummary Statistics:")
for factor in ['Sex', 'Historical lab', 'Country', 'Ivy league']:
    stats_result = compare_groups(df, factor)
    print(f"\n{factor}:")
    for group, median in zip(stats_result['groups'], stats_result['medians']):
        print(f"{group}: Median challenged rate = {median:.2f}%")
    print(f"p-value = {stats_result['p_value']:.3f}")

# Additional analysis for continuous relationships
if 'Continuity' in df.columns:
    correlation = stats.spearmanr(df['Continuity'], df['challenged_rate'])
    print("\nContinuity correlation:")
    print(f"Spearman correlation = {correlation.correlation:.3f}")
    print(f"p-value = {correlation.pvalue:.3f}")

# Save the figure
plt.savefig('challenged_claims_analysis.png', bbox_inches='tight', dpi=300)

# Create a summary table
summary_df = df.groupby(['Sex', 'Historical lab', 'Country']).agg({
    'challenged_rate': ['mean', 'median', 'std', 'count']
}).round(2)

print("\nDetailed Summary Table:")
print(summary_df)
